# 3D Vision Demo
In this notebook the methods of 3D vision are demonstrated

## Setup
### Install OpenCV

In [0]:
!pip install opencv-python
!wget https://github.com/jagracar/OpenCV-python-tests/raw/master/OpenCV-tutorials/data/tsukuba_l.png
!wget https://github.com/jagracar/OpenCV-python-tests/raw/master/OpenCV-tutorials/data/tsukuba_r.png

### Download Images

In [0]:
import requests
import os
import shutil

def save_file(name):
    url = "https://raw.githubusercontent.com/opencv/opencv/master/samples/data/" + name
    file = requests.get(url, stream=True)
    dump = file.raw
    location = os.path.abspath(name)
    with open(name, 'wb') as location:
        shutil.copyfileobj(dump, location)
        
for i in range(14):
    if i == 9:
        continue
    n = "left%02d.jpg" % (i + 1)
    save_file(n)
    n = "right%02d.jpg" % (i + 1)
    save_file(n)

## Camera Calibration

### Prepare point arrays

In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(7,5,0)
objp = np.zeros((6*8,3), np.float32)
objp[:,:2] = np.mgrid[0:8,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpointsL = [] # 2d points in left image plane.
imgpointsR = [] # 2d points in right image plane.

### Detect Chessboard corners

In [0]:
# Left and right images
imagesL = sorted(glob.glob('left*.jpg'))
imagesR = sorted(glob.glob('right*.jpg'))
        
plt.figure(figsize=(30,20))

for i,(fnameL,fnameR) in enumerate(zip(imagesL,imagesR)):
        
    # Read images
    imgL = cv2.imread(fnameL)
    imgR = cv2.imread(fnameR)
    
    # Grayscale
    grayL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)
    grayR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    retL, cornersL = cv2.findChessboardCorners(grayL, (8,6), cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_NORMALIZE_IMAGE+cv2.CALIB_CB_FAST_CHECK)
    retR, cornersR = cv2.findChessboardCorners(grayR, (8,6), cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_NORMALIZE_IMAGE+cv2.CALIB_CB_FAST_CHECK)
    
    # If found
    if retL and retR:
        
        # Subpixel refinement
        corners2L = cv2.cornerSubPix(grayL,cornersL, (11,11), (-1,-1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        corners2R = cv2.cornerSubPix(grayR,cornersR, (11,11), (-1,-1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        
        # Add object and image points to the lists
        objpoints.append(objp)
        imgpointsL.append(corners2L)
        imgpointsR.append(corners2R)
        
        # Draw
        cv2.drawChessboardCorners(imgL, (8,6), corners2L, retL)
        cv2.drawChessboardCorners(imgR, (8,6), corners2R, retR)
        
   
    plt.subplot(5,6,2*i+1)
    plt.imshow(imgL)
    plt.subplot(5,6,2*i+2)
    plt.imshow(imgR)
        

### Calibrate

In [0]:
retL, AL, distL, rvecsL, tvecsL = cv2.calibrateCamera(objpoints, imgpointsL, grayL.shape[::-1], None, cv2.CALIB_ZERO_TANGENT_DIST)
retR, AR, distR, rvecsR, tvecsR = cv2.calibrateCamera(objpoints, imgpointsR, grayR.shape[::-1], None, cv2.CALIB_ZERO_TANGENT_DIST)
print(retL,retR)
print(AL)
print(AR)
print(distL)
print(distR)

### Undistort

In [0]:
img = cv2.imread('left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(AL, distL, (w,h), 1, (w,h))
dst = cv2.undistort(img, AL, distL, None, newcameramtx)
plt.figure(figsize=(15,10))
plt.subplot(1,2,1)
plt.imshow(img)
plt.subplot(1,2,2)
plt.imshow(dst)

## Stereo Calibration
### Compute stereo calibration matrices

In [0]:
retS,_,_,_,_,R,T,E,F = cv2.stereoCalibrate(objpoints, imgpointsL, imgpointsR, AL, distL, AR, distR, (w,h))

print(retS)
print(R)
print(T)

### Compute Rectification transformation

In [0]:
RL,RR,PL,PR,Q,_,_ = cv2.stereoRectify(AL,distL,AR,distR,(w,h),R,T)

# Lookup table maps for rectification
mapL1,mapL2 = cv2.initUndistortRectifyMap(AL,distL,RL,PL,(w,h),cv2.CV_16SC2)
mapR1,mapR2 = cv2.initUndistortRectifyMap(AR,distR,RR,PR,(w,h),cv2.CV_16SC2)

### Apply rectification on the images

In [0]:
imgL = cv2.imread('left12.jpg')
imgR = cv2.imread('right12.jpg')

# Remap using Bilinear interpolation
uL = cv2.remap(imgL,mapL1,mapL2,cv2.INTER_LINEAR)
uR = cv2.remap(imgR,mapR1,mapR2,cv2.INTER_LINEAR)

# Put images side-by-side
und = np.concatenate((uL,uR),1)

plt.figure(figsize=(15,10))
plt.imshow(und)

## Disparity Computation

In [0]:
# Read stereo pair
imgL = cv2.imread('tsukuba_l.png',0)
imgR = cv2.imread('tsukuba_r.png',0)

# Create BM and SGBM algorithms
stereo = cv2.StereoBM_create(numDisparities=16, blockSize=7)
stereo2 = cv2.StereoSGBM_create(numDisparities=16, blockSize=7,P1=100,P2=300)

# Compute disparities
disparity = stereo.compute(imgL,imgR)
disparity2 = stereo2.compute(imgL,imgR)

# Plot
plt.figure(figsize=(20,20))
plt.subplot(1,3,1)
plt.imshow(imgL,'gray')
plt.subplot(1,3,2)
plt.imshow(disparity,'gray')
plt.subplot(1,3,3)
plt.imshow(disparity2,'gray')

### Reprojection

In [0]:
# Reproject
xyz = cv2.reprojectImageTo3D(disparity2,Q,handleMissingValues=True)

# Get X,Y,Z coordinates
xdata = xyz[...,0]
ydata = xyz[...,1]
zdata = xyz[...,2]

# Clean Outliers
xdata[zdata > 50] = 0
ydata[zdata > 50] = 0
zdata[zdata > 50] = 0

# 3D plotting
fig = plt.figure(figsize=(15,15))
plt.subplot(2,2,1)
plt.imshow(disparity2,'gray')
plt.subplot(2,2,2)
plt.imshow(xdata,'gray')
plt.subplot(2,2,3)
plt.imshow(ydata,'gray')
plt.subplot(2,2,4)
plt.imshow(zdata,'gray')